In [1]:
import mlflow

# Initialize MLFlow client
client = mlflow.tracking.MlflowClient()


# Register the model with a name, linking to the external model URI
model_details = client.create_registered_model(name="Shell1")

# Create a new version of the registered model pointing to the external URI
client.create_model_version(name="Shell1", source='', run_id=None)

print(f"Model version registered: {model_details}")

RestException: INVALID_PARAMETER_VALUE: Missing value for required parameter 'source'. See the API docs for more information about request parameters.

In [2]:
import os
import mlflow

# Directory where the empty model file will be created
model_dir = "empty_model"
os.makedirs(model_dir, exist_ok=True)

# Path to the empty model file
empty_model_file_path = os.path.join(model_dir, "empty_model.txt")

# Create an empty file
with open(empty_model_file_path, 'w') as f:
    f.write("")  # Write nothing to create an empty file

# Start an MLFlow run
with mlflow.start_run() as run:
    # Log the empty model file as an artifact
    mlflow.log_artifact(empty_model_file_path, artifact_path="model")

    # Register the model
    model_uri = f"runs:/{run.info.run_id}/model"
    mlflow.register_model(model_uri=model_uri, name="EmptyModel")

print("Empty model registered successfully.")

Successfully registered model 'EmptyModel'.
2025/01/22 14:03:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: EmptyModel, version 1
Created version '1' of model 'EmptyModel'.
2025/01/22 14:03:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run orderly-loon-48 at: http://127.0.0.1:8768/#/experiments/17/runs/0812d1f5bc354259a607efb67034cded.
2025/01/22 14:03:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8768/#/experiments/17.


Empty model registered successfully.


In [5]:
#
# Register ChatGPT endpoint
#

import mlflow.pyfunc
import requests

import pandas as pd
import requests
import mlflow.pyfunc

# Define the endpoint details
endpoint_url = "https://api.openai.com/v1/chat/completions"
api_key = os.environ['OPENAI_API_KEY']  #"<your_openai_api_key>"


class ChatGPTModelWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, endpoint_url, api_key):
        self.endpoint_url = endpoint_url
        self.api_key = api_key

    def predict(self, context, model_input):
        # Convert input to a list of strings (assumes DataFrame or Series input)
        if isinstance(model_input, pd.DataFrame):
            messages = model_input.iloc[:, 0].tolist()
        elif isinstance(model_input, pd.Series):
            messages = model_input.tolist()
        elif isinstance(model_input, list):
            messages = model_input
        else:
            raise ValueError("Unsupported input format for model_input")

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }
        payload = {
            "model": "gpt-4",
            "messages": [{"role": "user", "content": message} for message in messages],
        }
        response = requests.post(self.endpoint_url, headers=headers, json=payload)
        
        # Raise an error for bad HTTP responses
        response.raise_for_status()

        # Extract the API response
        return [choice["message"]["content"] for choice in response.json()["choices"]]

In [6]:
# Save the wrapped model to a local directory
model_path = "chatgpt_model"
mlflow.pyfunc.save_model(
    path=model_path,
    python_model=ChatGPTModelWrapper(endpoint_url, api_key),
)

MlflowException: Path 'chatgpt_model' already exists and is not empty

In [7]:
# Log and register the model
with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(
        artifact_path="chatgpt_model",
        python_model=ChatGPTModelWrapper(endpoint_url, api_key),
        registered_model_name="ChatGPT_Model",
    )

2025/01/22 14:06:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ChatGPT_Model'.
2025/01/22 14:06:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ChatGPT_Model, version 1
Created version '1' of model 'ChatGPT_Model'.
2025/01/22 14:06:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run incongruous-worm-774 at: http://127.0.0.1:8768/#/experiments/17/runs/b991956fa5494d54914124f1cbe9315d.
2025/01/22 14:06:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8768/#/experiments/17.


In [8]:
import mlflow.pyfunc
import pandas as pd

# Load the registered model
model = mlflow.pyfunc.load_model("models:/ChatGPT_Model/latest")

# Test predictions
input_data = pd.DataFrame(["Hello, how are you?"], columns=["prompt"])
result = model.predict(input_data)
print(result)

/opt/conda/lib/python3.10/site-packages/mlflow/store/artifact/utils/models.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


["As an artificial intelligence, I don't have feelings, but I'm here and ready to assist you. How can I help you today?"]
